In [28]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation, RepeatVector
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(0)

In [29]:
def n(digits=3):
    number = ''
    for i in range(np.random.randint(1, digits + 1)):
        number += np.random.choice(list('0123456789'))
    return int(number)


def padding(chars, maxlen):
    return chars + ' ' * (maxlen - len(chars))

In [30]:
'''
データの生成
'''
N = 20000
N_train = int(N * 0.9)
N_validation = N - N_train

digits = 3  # 最大の桁数
input_digits = digits * 2 + 1  # 例： 123+456
output_digits = digits + 1  # 500+500 = 1000 以上で４桁になる

added = set()
questions = []
answers = []

while len(questions) < N:
    a, b = n(), n()  # 適当な数を２つ生成

    pair = tuple(sorted((a, b)))
    if pair in added:
        continue

    question = '{}+{}'.format(a, b)
    question = padding(question, input_digits)
    answer = str(a + b)
    answer = padding(answer, output_digits)

    added.add(pair)
    questions.append(question)
    answers.append(answer)

chars = '0123456789+ '
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

X = np.zeros((len(questions), input_digits, len(chars)), dtype=np.integer)
Y = np.zeros((len(questions), digits + 1, len(chars)), dtype=np.integer)

for i in range(N):
    for t, char in enumerate(questions[i]):
        X[i, t, char_indices[char]] = 1
    for t, char in enumerate(answers[i]):
        Y[i, t, char_indices[char]] = 1

X_train, X_validation, Y_train, Y_validation = \
    train_test_split(X, Y, train_size=N_train)

In [4]:
'''
モデル設定
'''
n_in = len(chars)
n_hidden = 128
n_out = len(chars)

model = Sequential()

# Encoder
model.add(LSTM(n_hidden, input_shape=(input_digits, n_in)))

# Decoder
model.add(RepeatVector(output_digits))
model.add(LSTM(n_hidden, return_sequences=True))

model.add(TimeDistributed(Dense(n_out)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
              metrics=['accuracy'])

In [5]:
a = "aaaa"
b = "bbbb"
print ('a:', a, 'b:',b)

a: aaaa b: bbbb


In [26]:
'''
モデル学習
'''
epochs = 10
batch_size = 100

for epoch in range(epochs):
    print("epoch / epochs: ", (epoch + 1), "/", epochs)
    model.fit(X_train, Y_train, batch_size=batch_size, epochs=1,
              validation_data=(X_validation, Y_validation))

    # 検証データからランダムに問題を選んで答え合わせ
    for i in range(10):
        index = np.random.randint(0, N_validation)
        question = X_validation[np.array([index])]
        answer = Y_validation[np.array([index])]
        prediction = model.predict_classes(question, verbose=0)

        question = question.argmax(axis=-1)
        answer = answer.argmax(axis=-1)

        q = ''.join(indices_char[i] for i in question[0])
        a = ''.join(indices_char[i] for i in answer[0])
        p = ''.join(indices_char[i] for i in prediction[0])

        #print('-' * 10)
        #print('Q:  ', q, 'A:  ', p, 'T/F:', end=' ')
        #if a == p:
        #    print('TRUE')
        #else:
        #    print('FALSE')

epoch / epochs:  1 / 10
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 13s - loss: 1.9177e-04 - acc: 1.0000 - val_loss: 0.2700 - val_acc: 0.9425

In [25]:
from keras.models import load_model

model.save('encoder_decoder_model.h5')  # creates a HDF5 file 

In [31]:
'''
予測精度の評価
'''

questions_count = 1000
success_count = 0

for i in range(questions_count):
    index = np.random.randint(0, N_validation)
    question = X_validation[np.array([index])]
    answer = Y_validation[np.array([index])]
    prediction = model.predict_classes(question, verbose=0)

    question = question.argmax(axis=-1)
    answer = answer.argmax(axis=-1)

    q = ''.join(indices_char[i] for i in question[0])
    a = ''.join(indices_char[i] for i in answer[0])
    p = ''.join(indices_char[i] for i in prediction[0])

#    print('-' * 10)
 #   print('Q:  ', q, 'A:  ', p, 'T/F:', end=' ')
#        print('A:  ', p)
#        print('T/F:', end=' ')
    if a == p:
        success_count += 1 
#        print('TRUE')
    else:
        print('Q:  ', q, 'A:  ', p, 'T/F:', end=' ')
        print('FALSE')

accuracy = success_count / questions_count
print("success_count: ",success_count, "/",questions_count)
print("accuracy: ",accuracy)

Q:   101+447 A:   558  T/F: FALSE
Q:   537+3   A:   530  T/F: FALSE
Q:   371+446 A:   827  T/F: FALSE
Q:   47+823  A:   860  T/F: FALSE
Q:   727+526 A:   1143 T/F: FALSE
Q:   69+170  A:   249  T/F: FALSE
Q:   261+801 A:   162  T/F: FALSE
Q:   395+667 A:   1053 T/F: FALSE
Q:   520+912 A:   1422 T/F: FALSE
Q:   463+445 A:   809  T/F: FALSE
Q:   15+149  A:   174  T/F: FALSE
Q:   442+908 A:   1351 T/F: FALSE
Q:   502+1   A:   502  T/F: FALSE
Q:   864+249 A:   1103 T/F: FALSE
Q:   847+934 A:   1721 T/F: FALSE
Q:   836+282 A:   1108 T/F: FALSE
Q:   235+963 A:   1298 T/F: FALSE
Q:   375+780 A:   1145 T/F: FALSE
Q:   419+994 A:   1403 T/F: FALSE
Q:   462+38  A:   400  T/F: FALSE
Q:   552+7   A:   569  T/F: FALSE
Q:   334+174 A:   408  T/F: FALSE
Q:   82+837  A:   929  T/F: FALSE
Q:   408+95  A:   403  T/F: FALSE
Q:   603+19  A:   621  T/F: FALSE
Q:   393+300 A:   793  T/F: FALSE
Q:   727+526 A:   1143 T/F: FALSE
Q:   847+934 A:   1721 T/F: FALSE
Q:   25+925  A:   940  T/F: FALSE
Q:   579+416 A